## MultiAgent System
#### is a team that work together to achive a common goal

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from dotenv import load_dotenv
load_dotenv()
import os

key = os.getenv("Groq_api_key")

In [3]:
model_client =  OpenAIChatCompletionClient(
    base_url="https://api.groq.com/openai/v1",
    model="llama-3.3-70b-versatile",
    api_key = key,
    model_info={
        "family":'llama',
        "vision" :False,
        "function_calling":True,
        "json_output": True
    }
)

c:\Users\shubu\Desktop\AutoGen Hardcode\venv\lib\site-packages\autogen_ext\models\openai\_openai_client.py:413: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [4]:
single_story_agent = AssistantAgent(
    name="story_agent1",
    model_client=model_client,
    system_message="you are a creative write. generate a story on dog and a boy"
)

### Single agent approach to create a short story

In [6]:
async def test_single_agent():
    task = TextMessage(
        content="write a story on a dog and boy",
        source='user'
    )
    result = await single_story_agent.run(task = task)
    print(result.messages[-1].content)



await test_single_agent()   

**The Unbreakable Bond**

In a small village surrounded by rolling hills and lush green forests, there lived a young boy named Max. Max was a shy and introverted 10-year-old who had just moved to the village with his family. He found it difficult to make friends in his new school, and the loneliness was starting to get to him.

One day, while exploring the village, Max stumbled upon a small, adorable dog wandering around the streets. The dog was a golden retriever with a fluffy coat and big brown eyes that seemed to melt Max's heart. The dog was dirty and hungry, and Max could sense that it had been abandoned.

Without hesitation, Max decided to take the dog home and care for it. He named the dog Duke and quickly fell in love with his new companion. As the days went by, Max and Duke became inseparable. They would spend hours exploring the woods, playing fetch, and cuddling up by the fireplace.

As Max spent more time with Duke, he began to open up and feel more confident. Duke's friend

### Multi agent team approach to create a story

-  we'll use 3 agent to write it with their own mastery

In [7]:
plot_agent = AssistantAgent(
    name="plot_agent",
    model_client=model_client,
    system_message="you are a creative write. generate a story plot on a small dog and a small kid"
)

character_agent = AssistantAgent(
    name="character_agent",
    model_client=model_client,
    system_message="develope character of the story. describe dog and kid in detail with their background"
)

ending_agent = AssistantAgent(
    name="ending_agent",
    model_client=model_client,
    system_message="write engaging endings and end the story with a twist"
)

In [8]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [plot_agent, character_agent, ending_agent], # list of agents 
    max_turns=3
     # max_turns: 3, # if its 1 then only plot agent will work, if 2 then plot and character agent will work, 3 then all 3 will work
    # termination_condition: condition to terminate the agents
)


async def test_team():
    task = TextMessage(
        content="write story about dog and boy",
        source='user'
    )

    result = await team.run(task=task)
    # since its a team we will have multiple messages so 
    for each_agent_message in result.messages:
        print(f"{each_agent_message}: {each_agent_message.content}")

await test_team()

source='user' models_usage=None metadata={} content='write story about dog and boy' type='TextMessage': write story about dog and boy
source='plot_agent' models_usage=RequestUsage(prompt_tokens=59, completion_tokens=706) metadata={} content='**The Adventures of Max and Timmy**\n\nIn a small, sun-kissed town surrounded by green fields and blooming flowers, there lived a little boy named Timmy. Timmy was a curious and adventurous six-year-old who loved exploring the outdoors and making new friends. One day, his parents surprised him with a special gift - a small, fluffy dog named Max.\n\nMax was a playful and affectionate terrier mix with a coat of soft, golden fur and a wagging tail that never seemed to stop. From the moment Timmy laid eyes on Max, he knew they would be the best of friends. As soon as Max was settled into his new home, Timmy and he set out on their first adventure together.\n\nTheir backyard was a magical world, full of hidden treasures and secret paths waiting to be di